In [1]:
#!/.conda/envs/learn python
# -*- coding: utf-8 -*-

"""
多类别精度评定

预测结果和真值标签均为栅格且带有地理坐标系
~~~~~~~~~~~~~~~~
code by wHy
Aerospace Information Research Institute, Chinese Academy of Sciences
wanghaoyu191@mails.ucas.ac.cn
"""

'\n多类别精度评定\n\n预测结果和真值标签均为栅格且带有地理坐标系\n~~~~~~~~~~~~~~~~\ncode by wHy\nAerospace Information Research Institute, Chinese Academy of Sciences\nwanghaoyu191@mails.ucas.ac.cn\n'

In [2]:
import os
from statistics import mean
import sys
import fnmatch
import numpy as np
import gdal
import ogr
import osr
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [3]:
os.environ['GDAL_DATA'] = r'C:\Users\75198\.conda\envs\learn\Lib\site-packages\GDAL-2.4.1-py3.6-win-amd64.egg-info\gata-data' #防止报error4错误

ground_truth_path = r'E:\xinjiang_huyang_hongliu\Huyang_test_0808\1-raster_label_haze\huyang_haze_level_2_1.tif' # 存储真值标签的文件夹 真值标签应为栅格 带有地理坐标系
predict_path = r'E:\xinjiang_huyang_hongliu\Huyang_test_0808\3-predict_result\0-predict_result_Unet-huyang_real_haze_no_pretrain_230305\Talimu_result.tif' # 存储预测栅格的文件 带有地理坐标系

'''以真值标签为参考，从预测栅格文件中裁剪出待评定部分'''
# 获取真值标签地理坐标信息
input_small = gdal.Open(ground_truth_path)
geotransform_small = input_small.GetGeoTransform()
proj_small = input_small.GetProjection()
srs_small = osr.SpatialReference()
srs_small.ImportFromWkt(proj_small)

# 获取小的栅格影像的左上角和右下角地理坐标
xmin_small = geotransform_small[0]
ymax_small = geotransform_small[3]
xmax_small = geotransform_small[0] + geotransform_small[1] * input_small.RasterXSize
ymin_small = geotransform_small[3] + geotransform_small[5] * input_small.RasterYSize

# 打开大的栅格影像
input_large = gdal.Open(predict_path)

# 获取大的栅格影像的地理参考信息
geotransform_large = input_large.GetGeoTransform()
proj_large = input_large.GetProjection()
srs_large = osr.SpatialReference()
srs_large.ImportFromWkt(proj_large)

# 计算小的栅格影像在大的栅格影像中的位置
x_offset = int((xmin_small - geotransform_large[0]) / geotransform_large[1])
y_offset = int((geotransform_large[3] - ymax_small) / abs(geotransform_large[5]))

print(x_offset, y_offset)

# 定义裁剪窗口大小
win_size = input_small.RasterXSize

# 计算裁剪窗口范围
xmin = geotransform_large[0] + x_offset * geotransform_large[1]
ymax = geotransform_large[3] - y_offset * abs(geotransform_large[5])
xmax = xmin + win_size * geotransform_large[1]
ymin = ymax - win_size * abs(geotransform_large[5])

# 整理数据
im_data_pre = input_large.ReadAsArray(x_offset, y_offset, win_size, win_size)  # 读取预测结果对应区域的数据
im_data_true = input_small.ReadAsArray(0, 0, win_size, win_size) # 读取真值标签区域数据

print(im_data_pre.shape, im_data_true.shape)
print(type(im_data_pre))

im_data_pre = list(im_data_pre.reshape(-1)) # 展平为一维
im_data_true = list(im_data_true.reshape(-1)) # 展平为一维

6096 5736
(256, 256) (256, 256)
<class 'numpy.ndarray'>


In [4]:
'''精度评定部分'''
'''计算混淆矩阵'''
cm = confusion_matrix(im_data_true, im_data_pre)
print("Confusion matrix:")
print(cm, '\n')
accuracy = accuracy_score(im_data_true, im_data_pre)
balanced_accuracy = balanced_accuracy_score(im_data_true, im_data_pre)
precision = precision_score(im_data_true, im_data_pre, average='macro') # 'macro' 表示对所有类别的精确率求平均值
recall = recall_score(im_data_true, im_data_pre, average='macro') # 'macro' 表示对所有类别的召回率求平均值
f1 = f1_score(im_data_true, im_data_pre, average='macro')  # 'macro' 表示对所有类别的 F1 分数求平均值

# 输出综合精度指标
print('Accuracy:', accuracy)
print('Balanced Accuracy:', balanced_accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)
print('\n')

# 生成分类报告
report = classification_report(im_data_true, im_data_pre, target_names=['background', 'populus', 'red_willow'])
print(report)

Confusion matrix:
[[54671  1407   460]
 [  301  6675    89]
 [   76   207  1650]] 

Accuracy: 0.96124267578125
Balanced Accuracy: 0.9217905702340013
Precision: 0.8495922023570247
Recall: 0.9217905702340013
F1 Score: 0.882671646728515


              precision    recall  f1-score   support

  background       0.99      0.97      0.98     56538
     populus       0.81      0.94      0.87      7065
  red_willow       0.75      0.85      0.80      1933

    accuracy                           0.96     65536
   macro avg       0.85      0.92      0.88     65536
weighted avg       0.97      0.96      0.96     65536

